In [2]:
 %pip install -q langchain langchain-nvidia-ai-endpoints gradio langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [24]:
%pip install langserve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00


In [3]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

## 1. Chains and Runnables
- `RunnableLambda` -   Used to wrap a function (or a lambda) in such a way that the function can be "runnable" in a standardized way, meaning it can be executed as part of a pipeline or series of operations in a larger system.
- `RunnablePassThrough` -  It receives input and returns it exactly as it was, without any modifications. This can be useful when you want to include something in a pipeline or sequence of operations but don’t want to alter the data at that step.

In [4]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

identity = RunnableLambda(lambda x: x)

def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x

rprint0 = RunnableLambda(print_and_return)

In [5]:
## You can also pre-fill some of values using functools.partial
rprint1 = RunnableLambda(partial(print_and_return, preface="1: "))

## And you can use the same idea to make your own custom Runnable generator
def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

## Chaining two runnables
chain1 = identity | rprint0
chain1.invoke("Hello World!")
print()


Hello World!



In [6]:
## Chaining that one in as well
output = (
    chain1           ## Prints "Welcome Home!" & passes "Welcome Home!" onward
    | rprint1        ## Prints "1: Welcome Home!" & passes "Welcome Home!" onward
    | RPrint("2: ")  ## Prints "2: Welcome Home!" & passes "Welcome Home!" onward
).invoke("Welcome Home!")

## Final Output Is Preserved As "Welcome Home!"
print("\nOutput:", output)

Welcome Home!
1: Welcome Home!
2: Welcome Home!

Output: Welcome Home!


## 2. Simple LLM Chain

In [9]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Simple Chat Pipeline
chat_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])

rhyme_chain = prompt | chat_llm | StrOutputParser()

print(rhyme_chain.invoke({"input" : "Tell me about wisdom!"}))

Wisdom's a treasure so fine,
It's a quality that's truly divine.
It's gathered with years, or so they say,
Through life's experiences, day by day.

It's the voice of reason, the guide and the way,
To navigate life's ups and downs each new day.
It's the skill of discernment, a careful gaze,
To see through the noise, and find peaceful ways.

So cherish this wisdom, it's a precious find,
For it helps us grow, and leaves our fears behind.
With an open heart, and a mind so bright,
We can shine with wisdom, in the dark of night!


In [12]:
import gradio as gr

## Streaming Interface

def rhyme_chat_stream(message, history):
    buffer = ""
    for token in rhyme_chain.stream({"input" : message}):
        buffer += token
        yield buffer



In [13]:
## Uncomment when you're ready to try this.
demo = gr.ChatInterface(rhyme_chat_stream).queue()

window_kwargs = {}

demo.launch(share=True, debug=True, **window_kwargs)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fbe5ef0665caf34a6d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fbe5ef0665caf34a6d.gradio.live


## 3. Internal Response
In order, this zero-shot classification chain:

Takes in a dictionary with two required keys, input and options.
Passes it through the zero-shot prompt to get the input to our LLM.
Passes that string to the model to get the result.

In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

instruct_llm = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

## One-shot classification prompt with heavy format assumptions.
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_msg),
    ("user", "[[The sea is awesome]]"),
    ("assistant", "boat"),
    ("user", "[[{input}]]"),
])

In [16]:
## Roughly equivalent as above for <s>[INST]instruction[/INST]response</s> format
zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

zsc_chain = zsc_prompt | instruct_llm | StrOutputParser()


In [18]:
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

--------------------------------------------------------------------------------
car
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
airplane


## 4. Multi-Component Chain

In [19]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

## dictionary enforcement methods
def make_dictionary(v, key):
    if isinstance(v, dict):
        return v
    return {key : v}


def RInput(key='input'):
    '''Coercing method to mold a value (i.e. string) to in-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))



In [27]:
def ROutput(key='output'):
    '''Coercing method to mold a value (i.e. string) to out-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))


def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))


In [21]:
## Common LCEL utility for pulling values from dictionaries
from operator import itemgetter

up_and_down = (
    RPrint("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | RPrint("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | RPrint("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | RPrint("D: ")
    | itemgetter("word1")
    | RPrint("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | RPrint("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

up_and_down.invoke({"input" : "Hello World"})

A: {'input': 'Hello World'}
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}

In [22]:
## NOTE how the dictionary enforcement methods make it easy to make the following syntax equivalent
up_and_down.invoke("Hello World")

A: Hello World
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}

## 5. Deeper LangChain Integrations

In [ ]:
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# Ensure LangServe is running and accessible at the specified address and port
# Verify the address and port match your LangServe instance
llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()

for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')